In [8]:
from langchain.chat_models import ChatOpenAI
import os

import openai

from langchain.output_parsers import PydanticOutputParser


from langchain.pydantic_v1 import BaseModel, Field, validator
from langchain.prompts import PromptTemplate

In [9]:
os.environ['OPENAI_API_KEY'] = ''
openai.api_key = os.environ.get('OPENAI_API_KEY')

llm = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'])

In [18]:
# Define your desired data structure.

TRUE = ['yes', 'true']
FALSE = ['no', 'false']
class Binary(BaseModel):
    value: str = Field(description="yes or no value", examples=['yes', 'no'])

    def __bool__(self):
        if self.value in TRUE:
            return True
        else:
            return False

    def __str__(self):
        return self.value

binary_parser = PydanticOutputParser(pydantic_object=Binary)



    



template = "You are a helpful assistant in the field of {field} that search if the concept of {concept} is present or not in the text.\n {format_instructions}. \n The text is :  \n {text}\n"


neuro_concept_present_prompt = PromptTemplate(
    template=template,
    input_variables=["text", "concept"],
    partial_variables={"format_instructions": binary_parser.get_format_instructions(), "field":"neuroscience"},
)


In [41]:
from langchain.chains.llm import LLMChain
from langchain.output_parsers.openai_functions import (
    JsonOutputFunctionsParser,
    PydanticOutputFunctionsParser,
)

#function = _get_tagging_function(schema)
#llm_kwargs = get_llm_kwargs(function)

output_parser = JsonOutputFunctionsParser()

chain = LLMChain(
        llm=llm,
        prompt=neuro_concept_present_prompt,
        output_parser=output_parser,
        #llm_kwargs=llm_kwargs,
    )

In [42]:
chain.invoke({"concept":"despiking", "text":"I love despiking. It's help to understand the data"})

OutputParserException: Could not parse function call: 'function_call'

In [43]:
output_parser

JsonOutputFunctionsParser()

In [108]:
is_concept_present = neuro_concept_present_prompt | llm | binary_parser
small_chain = neuro_concept_present_prompt | llm

In [109]:
a = is_concept_present.invoke({"concept":"despiking", "text":"I love despiking. It's help to understand the data"})
a

Binary(value='yes')

In [21]:
str(a)

'no'

In [25]:
neuro_concept_present_prompt.invoke({"concept":"despiking", "text":"I love life. It's help to understand the data"}).text

'You are a helpful assistant in the field of neuroscience that search if the concept of despiking is present or not in the text.\n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"value": {"title": "Value", "description": "yes or no value", "examples": ["yes", "no"], "type": "string"}}, "required": ["value"]}\n```. \n The text is :  \n I love life. It\'s help to understand the data\n'

In [29]:
small_chain.invoke({"concept":"despiking", "text":"I love despiking. It's help to understand the data"})

AIMessage(content='{"value": "yes"}')

In [44]:


class ClusterMethod(BaseModel):
    method: int = Field(description="number of cluster", examples=[1,2,3])

clustering_parser = PydanticOutputParser(pydantic_object=ClusterMethod)


template = "You are a helpful assistant in the field of {field} that tell how many {method} there is.\n {format_instructions}. \n The text is :  \n {text}\n"


clustering_prompt = PromptTemplate(
    template=template,
    input_variables=["text", "method"],
    partial_variables={"format_instructions": clustering_parser.get_format_instructions(), "field":"neuroscience"},
)

In [45]:
cluster_method_use = clustering_prompt | llm | clustering_parser

In [46]:
a = cluster_method_use.invoke({"method":"clustering", "text":"We used 5 cluster."})
a

ClusterMethod(method=5)

In [73]:
type(cluster_method_use)

langchain_core.runnables.base.RunnableSequence

In [49]:
a.method

5

In [50]:
!pip install pypdf

   ---------------------------------------- 0.0/277.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/277.9 kB 435.7 kB/s eta 0:00:01
   ---- ---------------------------------- 30.7/277.9 kB 435.7 kB/s eta 0:00:01
   ---------------------------------------  276.5/277.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 277.9/277.9 kB 1.9 MB/s eta 0:00:00


In [51]:
from langchain.document_loaders import PyPDFLoader

In [57]:
loader = PyPDFLoader("2312.05250.pdf")

In [59]:
pages = loader.load_and_split()
pages[0]

Document(page_content='TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be aware of. For example, models solely trained to achieve\naccurate predictions may struggle to perform well on downstream tasks because\nseemingly small prediction errors may incur drastic task errors. The standard end-\nto-end learning approach is to make the task loss differentiable or to introduce a\ndifferentiable surrogate that the model can be trained on. In these settings, the task\nloss needs to be carefully balanced with the prediction loss because they may have\nconflicting objectives. We propose take the task loss signal one level deeper than the\nparameters of the model and use it to learn the parameters of the loss function the\nmodel is trained on, which can be done by learning a metric in the prediction space.\nThis ap

In [55]:
os.listdir()

['.ipynb_checkpoints', 'reviewer2go', 'Untitled.ipynb', 'Untitled1.ipynb']

In [63]:
pages[1].page_content

'Table 1: Settings we focus on where there is a discrepancy between the prediction task of a model\nand the downstream task where the model is deployed, i.e., Lpred̸=Ltask.\n(x) ( y) ( Lpred) ( Ltask)\nSetting Features Predictions Prediction Loss Task Loss\nPortfolio optimization Stock information Expected return of a stock MSE Portfolio’s performance\nBudget allocation Item information Value of item MSE Allocation’s performance\nModel-based RL Current state and action Next state MSE Value estimation given the model\nExamples of settings where the model’s prediction loss Lpredis mis-matched from the downstream\ntaskLtaskinclude the following, which table 1 also summarizes:\n1.theportfolio optimization setting from Wilder et al. [2019], which predicts the expected\nreturns from stocks for a financial portfolio. Here, the Lpredis the MSE and Ltaskis from the\nregret of running a portfolio optimization problem on the output;\n2.theallocation setting from Wilder et al. [2019], which predic

In [64]:
long_text = ' '.join(page.page_content for page in pages)

In [61]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [65]:
long_text

'TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be aware of. For example, models solely trained to achieve\naccurate predictions may struggle to perform well on downstream tasks because\nseemingly small prediction errors may incur drastic task errors. The standard end-\nto-end learning approach is to make the task loss differentiable or to introduce a\ndifferentiable surrogate that the model can be trained on. In these settings, the task\nloss needs to be carefully balanced with the prediction loss because they may have\nconflicting objectives. We propose take the task loss signal one level deeper than the\nparameters of the model and use it to learn the parameters of the loss function the\nmodel is trained on, which can be done by learning a metric in the prediction space.\nThis approach does not alter 

In [175]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)

In [176]:
texts = text_splitter.create_documents([long_text])
print(texts[0])
print(texts[1])
print(texts[2])
print(texts[3])

page_content='TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be aware of. For example, models solely trained to achieve\naccurate predictions may struggle to perform well on downstream tasks because\nseemingly small prediction errors may incur drastic task errors. The standard end-' metadata={'start_index': 0}
page_content='to-end learning approach is to make the task loss differentiable or to introduce a\ndifferentiable surrogate that the model can be trained on. In these settings, the task\nloss needs to be carefully balanced with the prediction loss because they may have\nconflicting objectives. We propose take the task loss signal one level deeper than the\nparameters of the model and use it to learn the parameters of the loss function the' metadata={'start_index': 447}
page_content='model is trained

In [101]:
texts[0].page_content

'TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be aware of. For example, models solely trained to achieve\naccurate predictions may struggle to perform well on downstream tasks because\nseemingly small prediction errors may incur drastic task errors. The standard end-\nto-end learning approach is to make the task loss differentiable or to introduce a\ndifferentiable surrogate that the model can be trained on. In these settings, the task\nloss needs to be carefully balanced with the prediction loss because they may have\nconflicting objectives. We propose take the task loss signal one level deeper than the\nparameters of the model and use it to learn the parameters of the loss function the\nmodel is trained on, which can be done by learning a metric in the prediction space.\nThis approach does not alter 

In [204]:
import json

def is_valid_json(my_json):
    try:
        json.loads(my_json)
        return True
    except ValueError as e:
        return False


class SubjectPresent:
    def __init__(self, text_spliter : RecursiveCharacterTextSplitter, model, parser):
        self.text_spliter = text_spliter
        self.model = model
        self.parser = parser

    def is_present(self, full_text:str, concept:str):
        texts = self.text_spliter.create_documents([full_text])
        for text in texts:
            print('a')
            is_present = self.model.invoke({"concept":concept, "text":text.page_content})
            if is_valid_json(is_present.content):
                if bool(self.parser.invoke(is_present)):
                    return True
            is_present = self.model.invoke({"concept":concept, "text":is_present.content})
            if is_valid_json(is_present.content):
                if bool(self.parser.invoke(is_present)):
                    return True
                    
        return False

            


In [205]:
basic_spliter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 500,
    chunk_overlap  = 20,
    length_function = len,
    add_start_index = True,
)

In [206]:
simple_chain = neuro_concept_present_prompt | llm 

is_present  = SubjectPresent(text_spliter = basic_spliter, model = simple_chain, parser = binary_parser)

In [207]:
is_present.is_present(full_text = long_text, concept = 'despiking')

a
a
a


True

In [113]:
a = 'You are a helpful assistant in the field of neuroscience that search if the concept of despiking is present or not in the text.\n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"value": {"title": "Value", "description": "yes or no value", "examples": ["yes", "no"], "type": "string"}}, "required": ["value"]}\n```. \n The text is :  \n TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be aware of. For example, models solely trained to achieve\naccurate predict'

In [155]:
a

'You are a helpful assistant in the field of neuroscience that search if the concept of despiking is present or not in the text.\n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"value": {"title": "Value", "description": "yes or no value", "examples": ["yes", "no"], "type": "string"}}, "required": ["value"]}\n```. \n The text is :  \n TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be a

In [179]:
simple_chain.invoke({"concept":"despiking", "text":texts[0].page_content})

AIMessage(content='The concept of despiking is not present in the given text.')

In [158]:
texts[0].page_content

'TaskMet: Task-Driven Metric Learning for Model Learning\nDishank Bansal∗Ricky T. Q. Chen Mustafa Mukadam Brandon Amos\nMeta\nAbstract\nDeep learning models are often deployed in downstream tasks that the training\nprocedure may not be aware of. For example, models solely trained to achieve\naccurate predictions may struggle to perform well on downstream tasks because\nseemingly small prediction errors may incur drastic task errors. The standard end-\nto-end learning approach is to make the task loss differentiable or to introduce a\ndifferentiable surrogate that the model can be trained on. In these settings, the task\nloss needs to be carefully balanced with the prediction loss because they may have\nconflicting objectives. We propose take the task loss signal one level deeper than the\nparameters of the model and use it to learn the parameters of the loss function the\nmodel is trained on, which can be done by learning a metric in the prediction space.\nThis approach does not alter 

In [178]:
simple_chain = neuro_concept_present_prompt | llm 

In [182]:
b = simple_chain.invoke({"concept":"despiking", "text":texts[0].page_content})

In [184]:
b.content

'{"value": "no"}'

In [186]:
binary_parser.invoke(b)

Binary(value='no')

Binary(value='no')